In [1]:
import grpc
import sys
import inference_pb2_grpc as pb2_grpc
import inference_pb2 as pb2

In [2]:
# Compile proto3 file
# python3 -m grpc_tools.protoc --proto_path=. ./inference.proto --python_out=. --grpc_python_out=.

In [2]:
class TestClient(object):
    """
    Client for testing inference gRPC
    """

    def __init__(self):
        #self.host = '3.139.238.241'
        self.host = 'localhost'
        self.server_port = 5125

        # instantiate a channel
        self.channel = grpc.insecure_channel(
            '{}:{}'.format(self.host, self.server_port))

        # bind the client and the server
        self.stub = pb2_grpc.InferenceStub(self.channel)

    def send(self, tx, modelHash, modelInput):
        """
        Client function to call the rpc for inference
        """
        message = pb2.InferenceParameters(tx=tx, modelHash=modelHash, modelInput = modelInput)
        print(f'{message}')
        return self.stub.RunInference(message)

def testLinearModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="LinearTest", modelInput="[[1.0, 2.0, 3.0], [2.5, 3.5, 4.5]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')

def testVolatilityModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="Volatility", modelInput="[[0.03],[0.05],[0.04056685],[0.03235871],[0.05629578]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')
    
def testXGBSpreadModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="XGBSpreadModel", modelInput="[[0.003], [0.005], [0.004056685]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')

def testLassoSpreadModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="QmXQpupTphRTeXJMEz3BCt9YUF6kikcqExxPdcVoL1BBhy", modelInput="[[0.003, 0.005, 0.004056685]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')

In [3]:
testVolatilityModel()

tx: "0x123"
modelHash: "Volatility"
modelInput: "[[0.03],[0.05],[0.04056685],[0.03235871],[0.05629578]]"



_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:5125: Failed to connect to remote host: Connection refused"
	debug_error_string = "UNKNOWN:failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:5125: Failed to connect to remote host: Connection refused {created_time:"2023-09-25T22:25:40.214703-04:00", grpc_status:14}"
>

In [4]:
testXGBSpreadModel()

tx: "0x123"
modelHash: "XGBSpreadModel"
modelInput: "[[0.003], [0.005], [0.004056685]]"



_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:5125: Failed to connect to remote host: Connection refused"
	debug_error_string = "UNKNOWN:failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:5125: Failed to connect to remote host: Connection refused {created_time:"2023-09-25T22:25:41.218556-04:00", grpc_status:14}"
>

In [10]:
testLassoSpreadModel()

tx: "0x123"
modelHash: "QmXQpupTphRTeXJMEz3BCt9YUF6kikcqExxPdcVoL1BBhy"
modelInput: "[[0.003, 0.005, 0.004056685]]"

Querying localhost:5125...
tx: "0x123"
node: "QmXQpupTphRTeXJMEz3BCt9YUF6kikcqExxPdcVoL1BBhy"
value: "0.001360592"

